In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"              # avoid DataParallel weirdness
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TRANSFORMERS_OFFLINE"] = "1"
import sys, torch
print("Python:", sys.executable)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
from dataclasses import dataclass
from typing import Any, Dict, List
from datasets import load_dataset
import torch
from transformers import (
    AutoProcessor, MllamaForConditionalGeneration,
    BitsAndBytesConfig, TrainingArguments
)
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from PIL import Image
import pandas as pd
from pathlib import Path
import json, random
import argparse
from datasets import load_dataset
from transformers import AutoProcessor, MllamaForConditionalGeneration, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from PIL import Image
import torch, os, re
from dataclasses import dataclass
import pandas as pd

In [ ]:
import os
from transformers import MllamaForConditionalGeneration, AutoProcessor
from PIL import Image
import torch

# Use your local model copy
LOCAL_DIR = "/home/hice1/wyiu31/scratch/models/llama32v"
os.environ["TRANSFORMERS_OFFLINE"] = "1"

# Safer default on shared GPUs (auto offload if VRAM is tight)
model = MllamaForConditionalGeneration.from_pretrained(
    LOCAL_DIR, device_map="auto", dtype=torch.float16, local_files_only=True
)
processor = AutoProcessor.from_pretrained(LOCAL_DIR, local_files_only=True)
print("Model & processor loaded")

In [ ]:
# ==== CONFIG (edit these) ====
BASE = "/home/hice1/wyiu31/scratch/models/llama32v"   # <-- base VLM, no FT
TRAIN_JSONL = "/home/hice1/wyiu31/scratch/llama32v_ft/data/version_11/train.jsonl"
VALID_JSONL = "/home/hice1/wyiu31/scratch/llama32v_ft/data/version_11/valid.jsonl"

# where to store per-phase CSVs (one per shard)
PARTS_DIR = "/home/hice1/wyiu31/scratch/llama32v_ft/base_eval_parts"
# final consolidated CSV (after all shards done)
FINAL_CSV = "/home/hice1/wyiu31/scratch/llama32v_ft/base_eval_full.csv"

PROMPT = "Identify the species; please give the common name."
BURST_WINDOW_SEC   = 90
MAX_IMGS_PER_BURST = 4

# how many shards you want to split the validation into
NUM_SHARDS = 5   # e.g. 3 phases
SHARD_IDX  = 0   # run 0 first, then 1, then 2...
OVERWRITE_PART = False   # False = resume this shard, True = start that shard fresh

import os, random, numpy as np, torch
os.environ.setdefault("TRANSFORMERS_OFFLINE", "1")

SEED = 1337
random.seed(SEED); np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import re, json, math, csv, time
from collections import defaultdict, Counter
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Any
from PIL import Image
from tqdm import tqdm

import torch
from transformers import AutoProcessor, MllamaForConditionalGeneration, BitsAndBytesConfig

TIMESTAMP_RE = re.compile(r"__(\d{8})_(\d{6})__")

# modest open-world bird expansion (added if missing)
BIRD_EXPANSION = [
    "Canada Goose", "Goose", "Snow Goose",
    "Mallard", "American Black Duck", "Wood Duck",
    "Ring-billed Gull", "Herring Gull",
    "Great Blue Heron", "Great Egret",
    "Mourning Dove", "Rock Pigeon",
    "Red-tailed Hawk", "Red-shouldered Hawk", "Barred Owl"
]

def norm_label(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"^common name[:\s]*", "", s, flags=re.I)
    s = re.sub(r"[^a-z0-9\s\-?']", "", s.lower())
    s = re.sub(r"\s+", " ", s).strip()
    return s

def ok_example(ex: Dict[str, Any]) -> bool:
    return (
        isinstance(ex.get("images"), list) and ex["images"] and
        isinstance(ex.get("output"), str) and ex["output"].strip()
    )

def parse_dt_from_path(p: str) -> datetime | None:
    m = TIMESTAMP_RE.search(p)
    if not m: return None
    try:
        return datetime.strptime(m.group(1)+m.group(2), "%Y%m%d%H%M%S")
    except Exception:
        return None

def burst_key(ex: Dict[str, Any]) -> tuple:
    md = ex.get("metadata") or {}
    loc = (md.get("location") or "").strip()

    dt = None
    imgs = ex.get("images") or []
    if imgs:
        dt = parse_dt_from_path(imgs[0])
    if dt is None:
        date_str = (md.get("date") or "").strip()
        time_str = (md.get("time") or "00:00:00").strip()
        try:
            if date_str and time_str:
                dt = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M:%S")
            elif date_str:
                dt = datetime.strptime(date_str, "%Y-%m-%d")
        except Exception:
            dt = None

    if dt is None:
        return (loc, "unknown", id(ex))

    bucket = int(dt.timestamp() // BURST_WINDOW_SEC)
    day = dt.strftime("%Y-%m-%d")
    return (loc, day, bucket)

def make_burst_id(key_tuple: tuple) -> str:
    loc, day, bucket = key_tuple
    return f"{loc}|{day}|{bucket}"

def merge_to_bursts(examples):
    groups = defaultdict(list)
    for ex in examples:
        groups[burst_key(ex)].append(ex)

    merged = []
    for key, items in groups.items():
        seen = set(); imgs = []
        for ex in items:
            for p in (ex.get("images") or []):
                if isinstance(p, str) and p not in seen:
                    imgs.append(p); seen.add(p)
        if not imgs:
            continue

        label = None
        for ex in items:
            out = ex.get("output")
            if isinstance(out, str) and out.strip():
                label = out.strip()
                break
        if not label:
            continue

        md0 = items[0].get("metadata") or {}
        merged.append({
            "burst_id": make_burst_id(key),
            "images": imgs[:MAX_IMGS_PER_BURST],
            "output": label,
            "metadata": {**md0, "burst_size": len(imgs), "burst_group_count": len(items)},
        })
    return merged

def load_candidates(*jsonls) -> List[str]:
    cnt = Counter()
    raw_map = {}
    for p in jsonls:
        if not os.path.exists(p): 
            continue
        with open(p) as f:
            for line in f:
                try:
                    ex = json.loads(line)
                except Exception:
                    continue
                out = ex.get("output", "")
                if not isinstance(out, str) or not out.strip():
                    continue
                raw = out.strip()
                nn = norm_label(raw)
                if not nn:
                    continue
                cnt[nn] += 1
                raw_map.setdefault(nn, raw)

    labels = [raw_map[nn] for nn,_ in cnt.items()]

    # add common birds if missing
    have_norm = {norm_label(x) for x in labels}
    for bird in BIRD_EXPANSION:
        if norm_label(bird) not in have_norm:
            labels.append(bird)

    return sorted(set(labels))

In [ ]:
# ---- load valid jsonl and build bursts ----
raw = []
with open(VALID_JSONL) as f:
    for line in f:
        try:
            ex = json.loads(line)
        except Exception:
            continue
        if ok_example(ex):
            raw.append(ex)

print(f"Loaded raw validation examples: {len(raw):,}")
bursts_all = merge_to_bursts(raw)
print(f"After burst merge (window={BURST_WINDOW_SEC}s): {len(bursts_all):,} bursts")

candidates = load_candidates(TRAIN_JSONL, VALID_JSONL)
print(f"Candidates: {len(candidates)} (sample: {candidates[:8]})")

In [ ]:
def load_base_model_and_processor(base_path: str):
    processor = AutoProcessor.from_pretrained(base_path, local_files_only=True)
    tok = processor.tokenizer
    if tok.pad_token_id is None and tok.eos_token_id is not None:
        tok.pad_token = tok.eos_token

    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    print("Loading BASE model only (no fine-tuned adapter)…")
    model = MllamaForConditionalGeneration.from_pretrained(
        base_path,
        quantization_config=bnb,
        device_map={"": 0} if device == "cuda" else None,
        local_files_only=True,
    )
    try:
        model.config._attn_implementation = "sdpa"
    except Exception:
        pass

    model.eval()
    model.config.use_cache = True
    if hasattr(model, "gradient_checkpointing_disable"):
        model.gradient_checkpointing_disable()
    return model, processor

model, processor = load_base_model_and_processor(BASE)

In [ ]:
def build_enc(processor, imgs: List[Image.Image]):
    msgs = [{
        "role": "user",
        "content": [*({"type":"image","image":im} for im in imgs),
                    {"type":"text","text": PROMPT}],
    }]
    txt = processor.apply_chat_template(
        msgs, add_generation_prompt=True, tokenize=True,
        return_tensors="pt", return_dict=True, padding=False, truncation=True
    )
    vis = processor(images=imgs, return_tensors="pt")
    enc = {**dict(txt), **dict(vis)}
    return {k: (v.to(device) if torch.is_tensor(v) else v) for k,v in enc.items()}

@torch.inference_mode()
def score_candidate(model, processor, enc, candidate_text: str) -> float:
    base_input_ids = enc["input_ids"]
    cand_ids = processor.tokenizer(candidate_text, return_tensors="pt", add_special_tokens=False)["input_ids"].to(device)
    input_ids = torch.cat([base_input_ids, cand_ids], dim=-1)
    attn = torch.cat([enc.get("attention_mask", torch.ones_like(base_input_ids)), torch.ones_like(cand_ids)], dim=-1)

    out = model(
        input_ids=input_ids,
        attention_mask=attn,
        pixel_values=enc.get("pixel_values"),
        aspect_ratio_mask=enc.get("aspect_ratio_mask"),
        aspect_ratio_ids=enc.get("aspect_ratio_ids"),
        use_cache=False,
        return_dict=True,
    )
    logits = out.logits[0]
    start = base_input_ids.shape[1] - 1
    end   = input_ids.shape[1] - 1
    if end <= start:
        return -1e9
    logp = torch.log_softmax(logits[start:end, :], dim=-1)
    target = input_ids[0, start+1:end+1].unsqueeze(-1)
    tok_logp = logp.gather(-1, target).squeeze(-1)
    return float(tok_logp.mean().item()) if tok_logp.numel() else -1e9

@torch.inference_mode()
def infer_burst_scoring(model, processor, image_paths: List[str], candidates: List[str]) -> str:
    imgs = []
    for p in image_paths:
        try:
            imgs.append(Image.open(p).convert("RGB"))
        except Exception:
            pass
    if not imgs:
        return "Unknown"

    enc = build_enc(processor, imgs)
    best_s, best_c = -1e9, "Unknown"
    for c in candidates:
        s = score_candidate(model, processor, enc, c)
        if s > best_s:
            best_s, best_c = s, c
    return best_c

def take_shard(items, num_shards: int, shard_idx: int):
    if num_shards <= 1:
        return list(items)
    return [items[i] for i in range(len(items)) if (i % num_shards) == shard_idx]

In [ ]:
parts_dir = Path(PARTS_DIR)
parts_dir.mkdir(parents=True, exist_ok=True)
part_path = parts_dir / f"base_part_{SHARD_IDX:02d}.csv"
header = ["burst_id","gold","pred","correct","images","burst_size"]

# get subset for this shard
bursts_shard = take_shard(bursts_all, NUM_SHARDS, SHARD_IDX)
print(f"Shard {SHARD_IDX}/{NUM_SHARDS}: {len(bursts_shard)} bursts")

# read existing rows if resuming
existing_ids = set()
if part_path.exists() and not OVERWRITE_PART:
    with part_path.open() as f:
        r = csv.DictReader(f)
        for row in r:
            existing_ids.add(row["burst_id"])

mode = "w" if (OVERWRITE_PART or not part_path.exists()) else "a"
f = part_path.open(mode, newline="")
w = csv.DictWriter(f, fieldnames=header)
if mode == "w":
    w.writeheader()

tot = ok = 0
per_class = defaultdict(lambda: {"n":0, "ok":0})

try:
    for ex in tqdm(bursts_shard, desc=f"Validate BASE (shard {SHARD_IDX})"):
        bid = ex["burst_id"]
        if bid in existing_ids:
            continue

        gold_raw = ex["output"]; gold = norm_label(gold_raw)
        pred_raw = infer_burst_scoring(model, processor, ex["images"], candidates)
        pred = norm_label(pred_raw)

        hit = int(pred == gold)
        tot += 1; ok += hit
        per_class[gold]["n"] += 1
        per_class[gold]["ok"] += hit

        row = {
            "burst_id": bid,
            "gold": gold_raw,
            "pred": pred_raw,
            "correct": hit,
            "images": "|".join(ex["images"]),
            "burst_size": ex.get("metadata",{}).get("burst_size", len(ex["images"])),
        }
        w.writerow(row)
        f.flush()   # important for long runs
finally:
    f.close()

acc = ok / max(1, tot)
print(f"\n[BASE Shard {SHARD_IDX}] Burst exact-match accuracy: {acc:.4f} ({ok}/{tot})")
top = sorted(per_class.items(), key=lambda kv: -kv[1]["n"])[:15]
print("\n[BASE Shard] Per-class (top 15):")
for cls, st in top:
    acc_c = (st["ok"] / st["n"]) if st["n"] else 0.0
    print(f"{cls:<30s}  {st['ok']:>4d}/{st['n']:<4d}  {acc_c:6.3f}")

print(f"\nShard written to: {part_path}")

In [ ]:
# Change the shard index each time (from 0 to 4)
SHARD_IDX = 1

In [ ]:
parts_dir = Path(PARTS_DIR)
parts_dir.mkdir(parents=True, exist_ok=True)
part_path = parts_dir / f"base_part_{SHARD_IDX:02d}.csv"
header = ["burst_id","gold","pred","correct","images","burst_size"]

# get subset for this shard
bursts_shard = take_shard(bursts_all, NUM_SHARDS, SHARD_IDX)
print(f"Shard {SHARD_IDX}/{NUM_SHARDS}: {len(bursts_shard)} bursts")

# read existing rows if resuming
existing_ids = set()
if part_path.exists() and not OVERWRITE_PART:
    with part_path.open() as f:
        r = csv.DictReader(f)
        for row in r:
            existing_ids.add(row["burst_id"])

mode = "w" if (OVERWRITE_PART or not part_path.exists()) else "a"
f = part_path.open(mode, newline="")
w = csv.DictWriter(f, fieldnames=header)
if mode == "w":
    w.writeheader()

tot = ok = 0
per_class = defaultdict(lambda: {"n":0, "ok":0})

try:
    for ex in tqdm(bursts_shard, desc=f"Validate BASE (shard {SHARD_IDX})"):
        bid = ex["burst_id"]
        if bid in existing_ids:
            continue

        gold_raw = ex["output"]; gold = norm_label(gold_raw)
        pred_raw = infer_burst_scoring(model, processor, ex["images"], candidates)
        pred = norm_label(pred_raw)

        hit = int(pred == gold)
        tot += 1; ok += hit
        per_class[gold]["n"] += 1
        per_class[gold]["ok"] += hit

        row = {
            "burst_id": bid,
            "gold": gold_raw,
            "pred": pred_raw,
            "correct": hit,
            "images": "|".join(ex["images"]),
            "burst_size": ex.get("metadata",{}).get("burst_size", len(ex["images"])),
        }
        w.writerow(row)
        f.flush()   # important for long runs
finally:
    f.close()

acc = ok / max(1, tot)
print(f"\n[BASE Shard {SHARD_IDX}] Burst exact-match accuracy: {acc:.4f} ({ok}/{tot})")
top = sorted(per_class.items(), key=lambda kv: -kv[1]["n"])[:15]
print("\n[BASE Shard] Per-class (top 15):")
for cls, st in top:
    acc_c = (st["ok"] / st["n"]) if st["n"] else 0.0
    print(f"{cls:<30s}  {st['ok']:>4d}/{st['n']:<4d}  {acc_c:6.3f}")

print(f"\nShard written to: {part_path}")

In [ ]:
# Change the shard index each time (from 0 to 4)
SHARD_IDX = 2

In [ ]:
parts_dir = Path(PARTS_DIR)
parts_dir.mkdir(parents=True, exist_ok=True)
part_path = parts_dir / f"base_part_{SHARD_IDX:02d}.csv"
header = ["burst_id","gold","pred","correct","images","burst_size"]

# get subset for this shard
bursts_shard = take_shard(bursts_all, NUM_SHARDS, SHARD_IDX)
print(f"Shard {SHARD_IDX}/{NUM_SHARDS}: {len(bursts_shard)} bursts")

# read existing rows if resuming
existing_ids = set()
if part_path.exists() and not OVERWRITE_PART:
    with part_path.open() as f:
        r = csv.DictReader(f)
        for row in r:
            existing_ids.add(row["burst_id"])

mode = "w" if (OVERWRITE_PART or not part_path.exists()) else "a"
f = part_path.open(mode, newline="")
w = csv.DictWriter(f, fieldnames=header)
if mode == "w":
    w.writeheader()

tot = ok = 0
per_class = defaultdict(lambda: {"n":0, "ok":0})

try:
    for ex in tqdm(bursts_shard, desc=f"Validate BASE (shard {SHARD_IDX})"):
        bid = ex["burst_id"]
        if bid in existing_ids:
            continue

        gold_raw = ex["output"]; gold = norm_label(gold_raw)
        pred_raw = infer_burst_scoring(model, processor, ex["images"], candidates)
        pred = norm_label(pred_raw)

        hit = int(pred == gold)
        tot += 1; ok += hit
        per_class[gold]["n"] += 1
        per_class[gold]["ok"] += hit

        row = {
            "burst_id": bid,
            "gold": gold_raw,
            "pred": pred_raw,
            "correct": hit,
            "images": "|".join(ex["images"]),
            "burst_size": ex.get("metadata",{}).get("burst_size", len(ex["images"])),
        }
        w.writerow(row)
        f.flush()   # important for long runs
finally:
    f.close()

acc = ok / max(1, tot)
print(f"\n[BASE Shard {SHARD_IDX}] Burst exact-match accuracy: {acc:.4f} ({ok}/{tot})")
top = sorted(per_class.items(), key=lambda kv: -kv[1]["n"])[:15]
print("\n[BASE Shard] Per-class (top 15):")
for cls, st in top:
    acc_c = (st["ok"] / st["n"]) if st["n"] else 0.0
    print(f"{cls:<30s}  {st['ok']:>4d}/{st['n']:<4d}  {acc_c:6.3f}")

print(f"\nShard written to: {part_path}")

In [ ]:
# Change the shard index each time (from 0 to 4)
SHARD_IDX = 3

In [ ]:
parts_dir = Path(PARTS_DIR)
parts_dir.mkdir(parents=True, exist_ok=True)
part_path = parts_dir / f"base_part_{SHARD_IDX:02d}.csv"
header = ["burst_id","gold","pred","correct","images","burst_size"]

# get subset for this shard
bursts_shard = take_shard(bursts_all, NUM_SHARDS, SHARD_IDX)
print(f"Shard {SHARD_IDX}/{NUM_SHARDS}: {len(bursts_shard)} bursts")

# read existing rows if resuming
existing_ids = set()
if part_path.exists() and not OVERWRITE_PART:
    with part_path.open() as f:
        r = csv.DictReader(f)
        for row in r:
            existing_ids.add(row["burst_id"])

mode = "w" if (OVERWRITE_PART or not part_path.exists()) else "a"
f = part_path.open(mode, newline="")
w = csv.DictWriter(f, fieldnames=header)
if mode == "w":
    w.writeheader()

tot = ok = 0
per_class = defaultdict(lambda: {"n":0, "ok":0})

try:
    for ex in tqdm(bursts_shard, desc=f"Validate BASE (shard {SHARD_IDX})"):
        bid = ex["burst_id"]
        if bid in existing_ids:
            continue

        gold_raw = ex["output"]; gold = norm_label(gold_raw)
        pred_raw = infer_burst_scoring(model, processor, ex["images"], candidates)
        pred = norm_label(pred_raw)

        hit = int(pred == gold)
        tot += 1; ok += hit
        per_class[gold]["n"] += 1
        per_class[gold]["ok"] += hit

        row = {
            "burst_id": bid,
            "gold": gold_raw,
            "pred": pred_raw,
            "correct": hit,
            "images": "|".join(ex["images"]),
            "burst_size": ex.get("metadata",{}).get("burst_size", len(ex["images"])),
        }
        w.writerow(row)
        f.flush()   # important for long runs
finally:
    f.close()

acc = ok / max(1, tot)
print(f"\n[BASE Shard {SHARD_IDX}] Burst exact-match accuracy: {acc:.4f} ({ok}/{tot})")
top = sorted(per_class.items(), key=lambda kv: -kv[1]["n"])[:15]
print("\n[BASE Shard] Per-class (top 15):")
for cls, st in top:
    acc_c = (st["ok"] / st["n"]) if st["n"] else 0.0
    print(f"{cls:<30s}  {st['ok']:>4d}/{st['n']:<4d}  {acc_c:6.3f}")

print(f"\nShard written to: {part_path}")

In [ ]:
# Change the shard index each time (from 0 to 4)
SHARD_IDX = 4

In [ ]:
parts_dir = Path(PARTS_DIR)
parts_dir.mkdir(parents=True, exist_ok=True)
part_path = parts_dir / f"base_part_{SHARD_IDX:02d}.csv"
header = ["burst_id","gold","pred","correct","images","burst_size"]

# get subset for this shard
bursts_shard = take_shard(bursts_all, NUM_SHARDS, SHARD_IDX)
print(f"Shard {SHARD_IDX}/{NUM_SHARDS}: {len(bursts_shard)} bursts")

# read existing rows if resuming
existing_ids = set()
if part_path.exists() and not OVERWRITE_PART:
    with part_path.open() as f:
        r = csv.DictReader(f)
        for row in r:
            existing_ids.add(row["burst_id"])

mode = "w" if (OVERWRITE_PART or not part_path.exists()) else "a"
f = part_path.open(mode, newline="")
w = csv.DictWriter(f, fieldnames=header)
if mode == "w":
    w.writeheader()

tot = ok = 0
per_class = defaultdict(lambda: {"n":0, "ok":0})

try:
    for ex in tqdm(bursts_shard, desc=f"Validate BASE (shard {SHARD_IDX})"):
        bid = ex["burst_id"]
        if bid in existing_ids:
            continue

        gold_raw = ex["output"]; gold = norm_label(gold_raw)
        pred_raw = infer_burst_scoring(model, processor, ex["images"], candidates)
        pred = norm_label(pred_raw)

        hit = int(pred == gold)
        tot += 1; ok += hit
        per_class[gold]["n"] += 1
        per_class[gold]["ok"] += hit

        row = {
            "burst_id": bid,
            "gold": gold_raw,
            "pred": pred_raw,
            "correct": hit,
            "images": "|".join(ex["images"]),
            "burst_size": ex.get("metadata",{}).get("burst_size", len(ex["images"])),
        }
        w.writerow(row)
        f.flush()   # important for long runs
finally:
    f.close()

acc = ok / max(1, tot)
print(f"\n[BASE Shard {SHARD_IDX}] Burst exact-match accuracy: {acc:.4f} ({ok}/{tot})")
top = sorted(per_class.items(), key=lambda kv: -kv[1]["n"])[:15]
print("\n[BASE Shard] Per-class (top 15):")
for cls, st in top:
    acc_c = (st["ok"] / st["n"]) if st["n"] else 0.0
    print(f"{cls:<30s}  {st['ok']:>4d}/{st['n']:<4d}  {acc_c:6.3f}")

print(f"\nShard written to: {part_path}")